In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, execute, QuantumCircuit
# Loading your IBM Q account(s)
import pylab
import numpy as np
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
import time
from qiskit.tools.monitor import job_monitor
from datetime import datetime
import sys
sys.path.insert(0, './')
from utility7 import *

from sklearn import linear_model
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.preprocessing import scale
import pickle

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub = "ibm-q", group='open', project='main')
backend = provider.get_backend('ibm_nairobi')#('ibmq_qasm_simulator')
backend.configuration()

ibmqfactory.load_account:WARNING:2022-11-13 20:23:26,152: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
#### data generating ####
N=1000
nrep = 100
snr=1
rho=0.7

for p in range(6,16):
    s=round(p/2)
    B = np.zeros((p,))
    B[0:s]=1
    Sigma = np.zeros((p,p))
    for i in range(p):
        for j in range(p):
            Sigma[i,j] = rho**abs(i-j)
    mu = np.zeros((p,))
    Input_list = []
    test_list = []
    for rep in range(nrep):
        np.random.seed(101+rep)
        X_train = np.random.multivariate_normal(mu, Sigma, N)+1
        epsion = np.random.normal(0,1,N)
        Y_train  = X_train.dot(B)+np.sqrt(B.dot(Sigma).dot(B)/snr)*epsion
        initial_candidate = [] # candidates of initial subset
        m = 5
        for i in range(m):
            initial_candidate.append([i for i in np.random.binomial(1, 0.5, p)])
        Input = dict({"Y_train":Y_train,"X_train":X_train,\
                    "initial_index":initial_candidate,"criterion": "Rsquared"})
        Input_list.append(Input)

    with open("./simu_Input_p%d"%p, "wb") as fp1:
        pickle.dump(Input_list, fp1)


In [4]:
for p in range(3,7):
    print(p)
    Input_list = []
    with open("./simu_Input_p%d"%p, 'rb') as fr3:
        Input_list.extend(pickle.load(fr3))
    experiment_res = []
    for Input in Input_list[0:10]:
        print("one replicate")
        experiment_res.append(BGS_linearReg_BIC(Input))
    with open("./IBM_res_simu_BGS_p%d"%p, "wb") as fp4:
        pickle.dump(experiment_res,fp4)


6
one replicate
Job Status: job has successfully run
24
Job Status: job has successfully run
24
Job Status: job has successfully run
22
Job Status: job has successfully run
22
Job Status: job has successfully run
22
Job Status: job has successfully run
one replicate
Job Status: job has successfully run
25
Job Status: job has successfully run
25
Job Status: job has successfully run
24
Job Status: job has successfully run
24
Job Status: job has successfully run
24
Job Status: job has successfully run
one replicate
Job Status: job has successfully run
31
Job Status: job has successfully run
31
Job Status: job has successfully run
31
Job Status: job has successfully run
31
Job Status: job has successfully run
one replicate
Job Status: job has successfully run
22
Job Status: job has successfully run
22
Job Status: job has successfully run
22
Job Status: job has successfully run
22
Job Status: job has successfully run
one replicate
Job Status: job has successfully run
30
Job Status: job has 

In [ ]:
time = []
time.extend(time3)
time.extend(time4)
time.extend(time5)
time.extend(time6)
time.extend(time7)
import matplotlib.pyplot as plt
import math
import seaborn as sns
D = 2**np.sort(np.array([i for i in range(3,8)]*10))
import pandas as pd
draw_data = pd.DataFrame({"time":time, "D":D})

In [ ]:
sns.set(rc={'figure.figsize':(6,4),'figure.dpi':600})
sns.set_style("whitegrid")
ax = sns.boxplot(y=time,x=D,data= draw_data,width = 0.5,color = "red")#plt.plot(2**np.array([3,4,5,6,7]),[time3,time4,time5,time6,time7],color = "red",label = "BGS")
x = np.arange(3,8)
ax.set_xticklabels([r"$2^{%d}$"%p for p in x])

ax = sns.lineplot(np.array([3,4,5,6,7])-3,1.5*2**(np.array([3,4,5,6,7])+3.5), linestyle='dashed',color = "black",label = "O(D)")
ax = sns.lineplot(np.array([3,4,5,6,7])-3,3*(np.array([3,4,5,6,7]))**2*np.sqrt(2**(np.array([3,4,5,6,7]))), linestyle='dashed',color = "blue",label = r"$O(\sqrt{D}(\log D)^3)$")
plt.xlabel("D")
plt.ylabel("Computational time in seconds")
plt.yscale('log', base=2)
plt.legend(loc = "upper left")
plt.savefig("res.jpg", dpi=300)